## PyTorch

torch.cuda - это пакет для поддержки CUDA. Он поддерживает такую же функциональность как и CPU, но использует CUDA ядра для вычислений. С полным функционалом можно ознакомиться [здесь](https://pytorch.org/docs/stable/cuda.html?highlight=cuda#module-torch.cuda)

In [ ]:
import torch

In [ ]:
print(f"Поддерживается ли CUDA : {torch.cuda.is_available()}")
print(f'Количество гпу девайсов: {torch.cuda.device_count()}')
print(f"Характеристики видеокарты : {torch.cuda.get_device_properties(0)}")

Давайте посмотрим, как работать с cuda. Допустим мы инициализуем два тензора:

In [ ]:
a = torch.normal(mean=torch.zeros(2, 4))
b = torch.normal(mean=torch.zeros(2, 4))
print(f"a:\n{a}\nb:\n{b}")

Наши тензоры автоматом загружены в память cpu. Но мы легко можем перевести их на cpu таким способом:

In [ ]:
a = a.cuda()
a

Теперь, если мы попробуем сложить эти два тензора, то у нас вылезет ошибка, т.к. один тензор на cpu, а другой на cuda:

In [ ]:
a + b

Мы не можем производить никакие операции с тензорами, находящимеся на разных устройствах. Что бы сложить их нам нужно оба тензора перевести на одно устройство:

In [ ]:
a + b.cuda()

In [ ]:
a.cpu() + b

In [ ]:
(a + b.cuda()).cpu()

Так же мы можем задать следующее определение устройства. Если  есть куда, то выбираем куду. В ином случае - цпу:

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

У каждого тензора есть поле device, которое по умолчанию стоит cpu. Но мы можем менять его при инициализации или в процессе использования:

In [ ]:
torch.randn(10, 10, device=device)

In [ ]:
a = torch.tensor((2 ,3))
print(a)

Переместить можно не только a.cuda(), но и так:

In [ ]:
a.to(device)

Но следует запомнить что .cuda() immutable функция. Т.е. она возвращает новый тензор, а не перезаписывает существующий a:

In [ ]:
a

Как видим наш тензор a все на том же cpu. Что бы интерпретатор запомнил что a у нас на куде необходимо присвоить значение выражение в тензор:

In [ ]:
a = a.cuda()

In [ ]:
a

Проверяем, находится ли сейчас тензор на куде:

In [ ]:
a.is_cuda

### Слои

Все основные модули которые будут рассматриваться ниже находятся в [torch.nn](https://pytorch.org/docs/stable/nn.html#). Все кроме оптимизаторов - они находятся в [torch.optim](https://pytorch.org/docs/stable/optim.html)

**Линейный слой (Линейное преобразование)**

[pytorch doc](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)

In [ ]:
layer = torch.nn.Linear(
    in_features=3,
    out_features=2,
    bias=True
)

In [ ]:
layer.weight, layer.bias

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1)
)
model

In [ ]:
model = torch.nn.Sequential()
model.add_module('L1', torch.nn.Linear(3, 1))
model

In [ ]:
import torchsummary

torchsummary.summary(model.to('cuda'), (3,))

### Алгоритм обучения в pytorch

In [ ]:
import torch
from torch import nn
from torch import optim

1. Для начала нам нужна модель через которую мы будем прогонять данные и получать какой-то результат. Для этого возьмем линейное преобразование:

In [ ]:
# torch.manual_seed(1)

model = nn.Sequential(
    nn.Linear(2, 1),
    nn.Sigmoid()
)

torchsummary.summary(model.to('cuda'), (2,))

У слоя в pytorch мы всегда можем посмотреть веса и отклонение:

In [ ]:
model.weight

In [ ]:
model[0], model[1]

In [ ]:
print('w: ', model[0].weight)
print('b: ', model[0].bias)

2. Теперь нам нужно определить функцию ошибок для подсчета градиента:

In [ ]:
criterion = nn.BCELoss()

3. Так же нам нужен оптимизатор который будет изменять веса нашей модели:

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

4. Нам нужны данные (х) и верные метки (y) по которым мы поймем правильные ли предсказания делает модель. (В данном случае у нас всего пара значений (данные, метки). О том как организовывать много данных ниже и в дальнейших вебинарах):

In [ ]:
x = torch.randn(2)
y = torch.zeros((1,))

x, y

5. Перед тем как считать градиенты и менять веса, нам нужно обнулить градиенты хранящиеся в свойстве тензора .grad. Для этого выполняем следующую строчку кода:

In [ ]:
optimizer.zero_grad()

6. Затем делаем предсказание на наших данных х, получаем предсказание модели и сохраняем это предсказание в переменную pred:

In [ ]:
x = x.to(device)
pred = model(x)

pred

7. Переменная pred имеет ту же размерность, что и y. y - это наша правильная метка (ground truth). На этом этапе мы сравниваем предсказанное с реальным и получаем некую численную оценку этого через функцию потерь:

In [ ]:
y = y.to(device)
loss = criterion(pred, y)
print('loss: ', loss, ' \nloss_item :', loss.item())

Стоит отметить, что если мы посмотрим на переменную .grad наших весов и отклонения, то ничего не будет:

In [ ]:
print('dL/dw: ', model[0].weight.grad)
print('dL/db: ', model[0].bias.grad)

Это потому что мы не начинали идти в обратном направлении и высчитывать градиенты.

8. Что ж, самое время это сделать. Проходим в обратном направлении и вычислим градиенты:

In [ ]:
loss.backward()

Теперь если мы посмотрим на grad весов, то уже что-то увидим:

In [ ]:
print('w: ', model[0].weight.grad)
print('b: ', model[0].bias.grad)

9. Теперь самое время поменять веса. Для этого надо сделать так называемый шаг оптимизатора. Здесь оптимизатор имея информацию о высчитанных градиентах и значениях весов меняет последние:

In [ ]:
# Веса до
print('BEFORE:\n','w: ', model[0].weight)
print('b: ', model[0].bias, '\n')

# Делаем шаг оптимизатора
optimizer.step()

# Веса после
print('AFTER:\n''w: ', model[0].weight)
print('b: ', model[0].bias)


Шаги обучения:
1. Проход по батчу
2. Обнуление градиента
3. Предсказание модели на батче
4. Подсчет ошибки
5. Подсчет градиентов
6. Шаг оптимизации
7. Логирование информации

### Создание сети через класс

Напишим небольшую полносвязную сеть на торче.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        # torch.manual_seed(1)
        super().__init__()
        self.fc = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # print('input', x)
        output = self.fc(x)
        # print('output fc', output)
        output = self.sigmoid(output)
        # print('output sigmoid', output)
        return output

In [ ]:
model = Net()
model = model.to(device)
model

In [ ]:
torchsummary.summary(model, (2,))

In [ ]:
model.fc.weight

In [ ]:
model['fc']

In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
optimizer.zero_grad()

In [ ]:
pred = model(x)
pred

In [ ]:
loss = criterion(pred, y)
print('loss: ', loss, ' \nloss_item :', loss.item())

In [ ]:
loss.backward()

In [ ]:
# Веса до
print('BEFORE:\n','w: ', model.fc.weight)
print('b: ', model.fc.bias, '\n')

# Делаем шаг оптимизатора
optimizer.step()

# Веса после
print('AFTER:\n''w: ', model.fc.weight)
print('b: ', model.fc.bias)

**Возьмем несколько объектов**

In [ ]:
x = torch.randn((3, 2))
y = torch.zeros((3,))

x = x.to(device)
y = y.to(device)
x, y

In [ ]:
pred = model(x)
pred

In [ ]:
loss = criterion(pred, y)

In [ ]:
loss = criterion(pred.squeeze(), y)
loss.item()

## Решение задачи классификации

In [ ]:
from sklearn.datasets import make_classification


x_train, y_train = make_classification(random_state=10,
                                       n_samples=40,
                                       n_features=2,
                                       n_informative=2,
                                       n_redundant=0)
X_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

x_test, y_test = make_classification(random_state=10,
                                     n_samples=10,
                                     n_features=2,
                                     n_informative=2,
                                     n_redundant=0,
                                     shuffle=True)
X_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(X_train.numpy()[:, 0], X_train.numpy()[:, 1], c=y_train.numpy())
plt.scatter(X_test.numpy()[:, 0], X_test.numpy()[:, 1], c=y_test.numpy(), marker='+');

#### 🧠 Упражнение. Обучение модели на бинарную классификацию

1. Создайте сеть из одного линейного слоя для задачи классификации

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = ...
        self.sigmoid = ...

    def forward(self, x):
        x = ...
        output = ...
        return output

2. Создайте объекты для подсчета функции потерь и для оптимизатора

In [ ]:
crit = ...
opt = ...

3. Реализуйте обучение 1000 эпох

- На одной эпохе обучайтесь на всей тренировочной выборке
- Каждые 10 эпох печатайте значение функции потерь на всей тренировочной и на всей тестовой выборках

In [ ]:
for epoch in range(1000):
    ...
    ...
    ...
    ...
    ...

    if epoch % 10 == 0:
        pred_test = ...
        loss_test = ...
        pred_train = ...
        loss_train = ...
        print(f'Iter {epoch}: train loss {loss_train.item():.2f} test loss {loss_test.item():.2f}')

4. После всего обучения посчитайте метрику accuracy на тесте и трейне

In [ ]:
pred_train = ...
pred_train_cls = ...

...

In [ ]:
pred_test = ...
pred_test_cls = ...

...

##### 🧠 Упражнение (ответ). Обучение модели на бинарную классификацию

1. Создайте сеть из одного линейного слоя для задачи классификации

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        output = self.fc(x)
        output = self.sigmoid(output)
        return output

In [ ]:
model = Net()
model

2. Создайте объекты для подсчета функции потерь и для оптимизатора

In [ ]:
crit = nn.BCELoss()
opt = optim.SGD(model.parameters(), lr=0.01)

3. Реализуйте обучение 1000 эпох

- На одной эпохе обучайтесь на всей тренировочной выборке
- Каждые 10 эпох печатайте значение функции потерь на всей тренировочной и на всей тестовой выборках


1. Проход по батчу
2. Обнуление градиента
3. Предсказание модели на батче
4. Подсчет ошибки
5. Подсчет градиентов
6. Шаг оптимизации
7. Логирование информации


In [ ]:
for epoch in range(1000):
    opt.zero_grad()
    pred = model(X_train)
    # loss = crit(pred, y_train)
    loss = crit(pred.squeeze(), y_train)
    loss.backward()
    opt.step()

    if epoch % 10 == 0:
        pred_test = model(X_test)
        loss_test = crit(pred_test.squeeze(), y_test)
        pred_train = model(X_train)
        loss_train = crit(pred_train.squeeze(), y_train)
        print(f'Iter {epoch}: train loss {loss_train.item():.2f} test loss {loss_test.item():.2f}')

4. После всего обучения посчитайте метрику accuracy на тесте и трейне

In [ ]:
pred_train = model(X_train)
pred_train_cls = torch.where(pred_train >= 0.5, 1, 0).squeeze()

(pred_train_cls == y_train).numpy().mean()

In [ ]:
pred_test = model(X_test)
pred_test_cls = torch.where(pred_test >= 0.5, 1, 0).squeeze()

(pred_test_cls == y_test).numpy().mean()

### Классификация изображений

Ссылка на датасет на каггл: https://www.kaggle.com/datasets/ashfakyeafi/glasses-classification-dataset/data

Ссылка на датасет на google drive: https://drive.google.com/file/d/1iC5c4pJwk-Wb07mP-Qr2tjcvJuRCnoOs

In [ ]:
!wget 'https://drive.google.com/uc?id=1iC5c4pJwk-Wb07mP-Qr2tjcvJuRCnoOs' -O glasses.zip

In [ ]:
!unzip glasses.zip

Здесь удобней не загружать все изображения в ОЗУ, а работать с ними через генератор

[ImageFolder](https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html).


In [ ]:
from torchvision.datasets import ImageFolder

train_data = ImageFolder('/content/train')

classes = train_data.classes
len(classes)

In [ ]:
train_data

In [ ]:
test_data = ImageFolder('/content/validate')

test_data

[DataLoader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#preparing-your-data-for-training-with-dataloaders) - объект для генерации батча изображений.

In [ ]:
import torch

batch_size = 32

train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=True)

In [ ]:
torch.manual_seed(1)
for data, y in train_loader:
    print(data.shape)
    print(y.shape)
    break

Нужно добавить преобразование изображений в тензоры. Для этого берем модуль [transforms](https://pytorch.org/vision/stable/transforms.html)

In [ ]:
from torchvision.transforms import transforms

transform = transforms.ToTensor()


# transform = transforms.Compose([
#     transforms.Resize((100, 100)),
#     # transforms.CenterCrop(64),
#     transforms.ToTensor()
# ])

In [ ]:
train_data = ImageFolder('/content/train', transform=transform)
test_data = ImageFolder('/content/validate', transform=transform)


train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=True)

In [ ]:
torch.manual_seed(1)
for data, y in train_loader:
    print(data.shape)
    print(y.shape)
    break

In [ ]:
y

In [ ]:
import matplotlib.pyplot as plt


torch.manual_seed(1)
ind = 3
for data, y in train_loader:
    plt.imshow(data[ind].permute(1, 2, 0))
    print(classes[y[ind]])
    break

In [ ]:
data[0].shape

In [ ]:
64 * 64 * 3

#### 🧠 Упражнение. Подготовка сети

- Постройте сеть с двумя линейными слоями
- Со слоем Flatten в начале
- С функцией активации ReLU на промежуточных слоях
- С размером скрытого представления 50

In [ ]:
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        torch.manual_seed(1)
        self.flat = ...
        self.fc1 = ...
        self.fc2 = ...
        self.relu =...

    def forward(self, x):
        ...
        return x



net = Net(64 * 64 * 3, 50, 2)
net = net.to(device)
net

In [ ]:
torchsummary.summary(net, (64, 64, 3))

##### 🧠 Упражнение (ответ). Подготовка сети

In [ ]:
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        torch.manual_seed(1)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.flat(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

    def predict(self, x):
        ...
        return x


net = Net(64 * 64 * 3, 50, 2)
net = net.to(device)
net

In [ ]:
torchsummary.summary(net, (64, 64, 3))

### Обучение модели

In [ ]:
example = data[:1].to(device)
example.shape

In [ ]:
pred = net(example)
pred

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    running_loss = 0.0
    running_items = 0.0
    running_correct = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_items += len(labels)
        _, predicted = torch.max(outputs, 1)
        running_correct += (predicted == labels).sum().item()

    if epoch % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
              f'Loss: {running_loss / running_items:.3f}. ' \
              f'Acc: {running_correct / running_items:.3f}')
        running_loss, running_items, running_correct = 0.0, 0.0, 0.0

In [ ]:
pred = net(example)
pred
# nn.Softmax()(pred)

#### Сохранение модели

In [ ]:
PATH_WEIGHTS = 'net_weights.pth'
torch.save(net.state_dict(), PATH_WEIGHTS)

In [ ]:
print("Model state_dict: ")
for param in net.state_dict():
    print(param, "\t", net.state_dict()[param].size())

In [ ]:
PATH_MODEL = 'net.pth'
torch.save(net, PATH_MODEL)

#### Загрузка и использование модели

In [ ]:
net2 = Net(64 * 64 * 3, 50, 2)
net2.load_state_dict(torch.load(PATH_WEIGHTS))
net2

In [ ]:
net2 = torch.load(PATH_MODEL)

In [ ]:
torch.manual_seed(1)

for imgs, y in test_loader:
    break

imgs = imgs.to(device)[:8]
y = y.to(device)[:8]
imgs.shape, y.shape

In [ ]:
import torchvision

outputs = net(imgs)
imgs = torchvision.utils.make_grid(imgs).cpu()
plt.figure(figsize=(10, 5))
plt.imshow(imgs.permute(1, 2, 0).numpy())

In [ ]:
print(outputs)

In [ ]:
_, predicted = torch.max(outputs, 1)

predicted

In [ ]:
import numpy as np

gt = np.array([classes[labels[j]] for j in range(len(labels))])
pred = np.array([classes[predicted[j]] for j in range(len(labels))])

print(gt)
print(pred)
print(f'Accuracy is {(gt == pred).sum() / len(gt)}')

## Дополнительные материалы
1. Официальная документация PyTorch https://pytorch.org/tutorials/
2. Метод обратного распространения ошибки https://youtu.be/EuhoXsuu8SQ
3. Функции активаций https://youtu.be/Gs8T_qF-FAA
